In [1]:
!git clone https://github.com/katipallyvig8899/ERAS15.git

Cloning into 'ERAS15'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (10/10), 9.70 KiB | 1.94 MiB/s, done.


In [2]:
cd /kaggle/working/ERAS15

/kaggle/working/ERAS15


In [3]:
!pip install -r requirements.txt


In [4]:
!python model.py

In [5]:
!python config.py

In [6]:
import config
import dataset
import utils
import dataloader

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule
from torch.optim.lr_scheduler import LambdaLR
import torchmetrics
from torchmetrics import BLEUScore, CharErrorRate, WordErrorRate

from model import build_transformer
from utils import causal_mask
import pytorch_lightning as pl
from config import get_config
from dataloader import OpusDataSetModule
config = get_config()



class BilingualModel(LightningModule):
    def __init__(self, learning_rate, tokenizer_src, tokenizer_tgt, config=config):

        super().__init__()

        self.learning_rate = learning_rate
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt

        self.seq_len = config["seq_len"]

        self.src_vocab_size = tokenizer_src.get_vocab_size()
        self.tgt_vocab_size = tokenizer_tgt.get_vocab_size()

        # self.src_vocab_size = 15698
        # self.tgt_vocab_size = 22463

        self.last_batch = None

        self.network = build_transformer(
            self.src_vocab_size,
            self.tgt_vocab_size,
            self.seq_len,
            self.seq_len,
            d_model=config["d_model"],
        )

        self.loss_fn = nn.CrossEntropyLoss(
            ignore_index=self.tokenizer_src.token_to_id("[PAD]"), label_smoothing=0.1
        )

        self.epoch_condition = 0

        self.train_loss = []
        self.val_loss = []

        self.source_texts = []
        self.expected = []
        self.predicted = []

        self.char_error_rate = CharErrorRate()
        self.word_error_rate = WordErrorRate()
        self.bleu_score = BLEUScore()

    def forward(self, encoder_input, decoder_input, encoder_mask, decoder_mask):
        encoder_output = self.network.encode(
            encoder_input, encoder_mask
        )  # (B, seq_len, seq_len)
        decoder_output = self.network.decode(
            encoder_output, encoder_mask, decoder_input, decoder_mask
        )
        proj_output = self.network.project(decoder_output)  # (B, seq_len, vocab_size)

        return proj_output

    def training_step(self, batch, batch_idx):
        encoder_input = batch["encoder_input"]
        decoder_input = batch["decoder_input"]
        encoder_mask = batch["encoder_mask"]
        decoder_mask = batch["decoder_mask"]
        proj_output = self.forward(
            encoder_input, decoder_input, encoder_mask, decoder_mask
        )

        label = batch["label"]  # (B, seq_len)
        loss = self.loss_fn(
            proj_output.view(-1, self.tokenizer_tgt.get_vocab_size()), label.view(-1)
        )

        # update and log metrics
        self.train_loss.append(loss)
        mean_loss = sum(self.train_loss) / len(self.train_loss)
        self.log("Train Loss", mean_loss, prog_bar=True)

        self.epoch_condition = 1

        return loss

    def validation_step(self, batch, batch_idx):
        encoder_input = batch["encoder_input"]
        decoder_input = batch["decoder_input"]
        encoder_mask = batch["encoder_mask"]
        decoder_mask = batch["decoder_mask"]
        proj_output = self.forward(
            encoder_input, decoder_input, encoder_mask, decoder_mask
        )

        label = batch["label"]  # (B, seq_len)
        loss = self.loss_fn(
            proj_output.view(-1, self.tokenizer_tgt.get_vocab_size()), label.view(-1)
        )

        # update and log metrics
        self.val_loss.append(loss)
        mean_loss = sum(self.val_loss) / len(self.val_loss)
        self.log("Val Loss", mean_loss, prog_bar=True)

        self.last_batch = batch
        return loss

    def on_validation_epoch_end(self) -> None:
        if self.epoch_condition > 0:

            print(f"Epoch : {self.current_epoch}")

            train_mean_loss = sum(self.train_loss) / len(self.train_loss)
            print(f"Training Loss : {train_mean_loss:5f}")

            val_mean_loss = sum(self.val_loss) / len(self.val_loss)
            print(f"Validation Loss : {val_mean_loss:5f}")

            self.run_validation(self, self.last_batch)

            self.train_loss = []
            self.val_loss = []

            self.epoch_condition = 0

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, eps=1e-9)

        return [optimizer]

    def greedy_decode(self, source, source_mask, max_len: int, device: str):
        sos_idx = self.tokenizer_tgt.token_to_id("[SOS]")
        eos_idx = self.tokenizer_tgt.token_to_id("[EOS]")

        # encoder output
        encoder_output = self.network.encode(source, source_mask)
        decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
        while True:
            if decoder_input.size(1) == max_len:
                break
            # build target mask
            decoder_mask = (
                causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)
            )
            out = self.network.decode(
                encoder_output, source_mask, decoder_input, decoder_mask
            )

            # get next token
            prob = self.network.project(out[:, -1])
            _, next_word = torch.max(prob, dim=1)
            decoder_input = torch.cat(
                [
                    decoder_input,
                    torch.empty(1, 1)
                    .type_as(source)
                    .fill_(next_word.item())
                    .to(device),
                ],
                dim=1,
            )
            if next_word == eos_idx:
                break

        return decoder_input.squeeze(0)

    def run_validation(self, model, data):
        model.eval()

        src = data["encoder_input"]
        src_mask = data["encoder_mask"]

        model_out = self.greedy_decode(
            src, src_mask, max_len=self.seq_len, device=self.device
        )

        source_text = data["src_text"]
        target_text = data["tgt_text"]
        model_out_text = self.tokenizer_tgt.decode(model_out.detach().cpu().numpy())

        expected = target_text
        predicted = model_out_text

        metric = CharErrorRate()
        cer = metric(predicted, expected)

        metric = WordErrorRate()
        wer = metric(predicted, expected)

        metric = BLEUScore()
        bleu = metric(predicted, expected)

        print("----------------------------------------------------------------------")
        print(f"SOURCE    => {source_text}")
        print(f"Ground Truth  => {expected}")
        print(f"PREDICTED => {predicted}")
        print("----------------------------------------------------------------------")
        print(f"Validation CER  => {cer}")
        print(f"Validation WER  => {wer}")
        print(f"Validation BLEU => {bleu}")
        print("----------------------------------------------------------------------")

        self.log("Validation - CER", cer, prog_bar=True)
        self.log("Validation - WER", wer, prog_bar=True)
        self.log("Validation - BLEU", bleu, prog_bar=True)

        model.train()


data = OpusDataSetModule()
data.setup()

src, tgt = data.tokenizer_src, data.tokenizer_tgt
model = BilingualModel(learning_rate=1e-4, tokenizer_src=src, tokenizer_tgt=tgt)

trainer = pl.Trainer(precision=16, max_epochs=config['num_epochs'], accelerator="gpu")

trainer.fit(model, data)



Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Dataset opus_books downloaded and prepared to /root/.cache/huggingface/datasets/opus_books/en-it/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf. Subsequent calls will reuse this data.
Max length of source sentence: 309
Max length of target sentence: 274
Source Tokenizer Vocab Size : 15698
Target Tokenizer Vocab Size : 22463




/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `CharErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CharErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `WordErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `WordErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `BLEUScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `BLEUScore` from `torchmetrics.text` instead.
  _future_warning(
/opt/conda/lib/python3.10/site-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


...

Epoch : 7
Training Loss : 4.018532
Validation Loss : 4.581522
----------------------------------------------------------------------
SOURCE    => ['A letter?']
Ground Truth  => ['Una lettera?']
PREDICTED => Un ’ lettera ?
----------------------------------------------------------------------
Validation CER  => 0.25
Validation WER  => 2.0
Validation BLEU => 0.0
----------------------------------------------------------------------


Validation: 0it [00:00, ?it/s]

Epoch : 8
Training Loss : 3.794404
Validation Loss : 4.613221
----------------------------------------------------------------------
SOURCE    => ['A letter?']
Ground Truth  => ['Una lettera?']
PREDICTED => Un ’ lettera ?
----------------------------------------------------------------------
Validation CER  => 0.25
Validation WER  => 2.0
Validation BLEU => 0.0
----------------------------------------------------------------------


Validation: 0it [00:00, ?it/s]

Epoch : 9
Training Loss : 3.579368
Validation Loss : 4.648872
----------------------------------------------------------------------
SOURCE    => ['A letter?']
Ground Truth  => ['Una lettera?']
PREDICTED => Una lettera ?
----------------------------------------------------------------------
Validation CER  => 0.0833333358168602
Validation WER  => 1.0
Validation BLEU => 0.0
----------------------------------------------------------------------


In [8]:
# start tensorboard
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/